In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [ ]:
# Read Spotify songs data from CSV file
songs = pd.read_csv("raw/spotify_songs.csv")

# Read Spotify user playlist data from CSV file, handle bad lines by skipping them, and name the columns explicitly
playlist = pd.read_csv("raw/spotify_dataset.csv", on_bad_lines='skip', names=["user_id", "artist_name", "track_name", "playlist_name"])

# Read chart data from CSV file
charts = pd.read_csv("raw/charts.csv")

In [ ]:
# Convert 'date' column to datetime format in the charts DataFrame
charts["date"] = pd.to_datetime(charts.date)

In [ ]:
# Rename columns in the songs DataFrame for consistency
songs = songs.rename(columns={"track_artist": "artist_name"})

# Rename columns in the charts DataFrame for consistency
charts = charts.rename(columns={"artist": "artist_name", "title": "track_name"})

In [ ]:
# Filter charts data to only include entries from the 1st day of months that are divisible by 6 (June and December)
charts = charts[(charts["date"].dt.day == 1) & (charts["date"].dt.month % 6 == 0)]

In [ ]:
# Artist table

In [ ]:
# Create a DataFrame for artists, extracting unique artist names and assigning them an index as 'artist_id'
artists = pd.DataFrame(songs.artist_name.unique(), columns=["artist_name"])
artists.loc[:, "artist_id"] = artists.index

In [ ]:
# Create a mapping from artist names to artist IDs
mapping = artists.set_index("artist_name")

# Initialize 'artist_id' column in songs DataFrame
songs.loc[:, "artist_id"] = pd.Series(dtype=int)

# Map artist names to artist IDs in songs DataFrame using the mapping created earlier
songs.loc[:, "artist_id"] = songs.artist_name.apply(lambda x: mapping.loc[x])

In [ ]:
# Intersection of datasets

In [ ]:
# Define keys and IDs for merging DataFrames
merge_keys = ["track_name", "artist_name"]
merge_ids = ["track_id", "artist_id"]

In [ ]:
# Merge playlist with songs data to create a DataFrame that includes user playlists with song and artist IDs
playlist_user = playlist.merge(songs[[*merge_keys, *merge_ids]], on=merge_keys).reset_index(drop=True)
playlist_user.loc[:, "playlist_user_id"] = playlist_user.index

In [ ]:
# Combine Spotify and user playlists

In [ ]:
# Prepare columns to be included from the songs DataFrame for the Spotify playlists DataFrame
playlist_cols = ["playlist_id", "playlist_name", "playlist_genre", "playlist_subgenre"]

# Create a DataFrame for Spotify playlists, assigning new IDs for Spotify playlists
playlist_spotify = songs[[*merge_keys, *merge_ids, *playlist_cols]].rename(columns={"playlist_id": "playlist_spotify_id"})
songs = songs.drop(columns=playlist_cols)
playlist_spotify.loc[:, "playlist_spotify_id"] = playlist_spotify.index

In [ ]:
# Merge charts data with songs data
charts = charts.merge(songs[[*merge_keys, *merge_ids]], on=merge_keys)

In [ ]:
# Create separate DataFrames for Viral50 and Top200 charts and drop unneeded columns
chart_drop_cols = ["chart", "track_name", "artist_name", "url"]
viral50 = charts[charts['chart'] == 'viral50'].drop(columns=chart_drop_cols).reset_index(drop=True)
viral50.loc[:, "viral50_id"] = viral50.index
top200 = charts[charts['chart'] == 'top200'].drop(columns=chart_drop_cols).reset_index(drop=True)
top200.loc[:, "top200_id"] = top200.index

In [ ]:
# Create Facts Table

In [ ]:
# Create a Facts Table by merging playlist and chart DataFrames on song and artist IDs
playlist_merged = (
    playlist_user[[*merge_ids, "user_id", "playlist_user_id"]]
    .merge(playlist_spotify[[*merge_ids, "playlist_spotify_id"]], on=merge_ids, how="outer")
)

In [ ]:
charts_merged = (
    viral50[[*merge_ids, "viral50_id"]]
    .merge(top200[[*merge_ids, "top200_id"]], on=merge_ids, how="outer")
)

In [ ]:
facts = playlist_merged.merge(charts_merged, on=merge_ids, how="outer")

In [ ]:
# Export processed data to CSV files
songs.to_csv("processed/songs.csv", index=False)
artists.to_csv("processed/artists.csv", index=False)
playlist_user.to_csv("processed/playlist_user.csv", index=False)
playlist_spotify.to_csv("processed/playlist_spotify.csv", index=False)
viral50.to_csv("processed/viral50.csv", index=False)
top200.to_csv("processed/top200.csv", index=False)
facts.to_csv("processed/facts.csv", index=False)